<a href="https://colab.research.google.com/github/AycaCicek/DogrusalOlmayanRegresyonModelleri/blob/main/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import scale,StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [2]:
data=pd.read_csv("/content/Hitters.csv")
data.head(3)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A


In [3]:
data=data.dropna()
data_encoding=pd.get_dummies(data[["League","Division","NewLeague"]])
data_encoding.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [4]:
#--> bağımlı değişken : 
y = data["Salary"]
#--> Bağımlı DEğişken : 
X_=data.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
X = pd.concat([X_,data_encoding[["League_N","Division_W","NewLeague_N"]]],axis=1)
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.25,random_state=42)

Model & Tahmin

In [6]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 149 kB/s 


In [7]:
from catboost import CatBoostRegressor

In [8]:
catb=CatBoostRegressor().fit(X_train,y_train)

Learning rate set to 0.026584
0:	learn: 361.4500620	total: 46.9ms	remaining: 46.9s
1:	learn: 356.9623048	total: 48.3ms	remaining: 24.1s
2:	learn: 353.1516670	total: 49.3ms	remaining: 16.4s
3:	learn: 348.9660594	total: 50.4ms	remaining: 12.6s
4:	learn: 345.4052783	total: 51.2ms	remaining: 10.2s
5:	learn: 341.9084513	total: 52ms	remaining: 8.61s
6:	learn: 338.3607714	total: 52.7ms	remaining: 7.48s
7:	learn: 334.9255912	total: 53.5ms	remaining: 6.63s
8:	learn: 331.3397385	total: 54.2ms	remaining: 5.97s
9:	learn: 328.2861219	total: 55ms	remaining: 5.44s
10:	learn: 325.4275763	total: 55.7ms	remaining: 5.01s
11:	learn: 322.1296399	total: 56.7ms	remaining: 4.67s
12:	learn: 319.0063786	total: 57.8ms	remaining: 4.38s
13:	learn: 315.3788301	total: 58.7ms	remaining: 4.14s
14:	learn: 312.3594813	total: 59.8ms	remaining: 3.93s
15:	learn: 309.4918933	total: 60.9ms	remaining: 3.74s
16:	learn: 306.5173079	total: 61.9ms	remaining: 3.58s
17:	learn: 303.6252680	total: 62.9ms	remaining: 3.43s
18:	learn: 3

In [9]:
catb.get_params()

{'loss_function': 'RMSE'}

In [10]:
y_pred=catb.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

338.9183080950738

In [11]:
params={"iterations": [200,500,1000],
        "learning_rate" : [0.01,0.1],
        "depth": [3,6,8]}

In [12]:
catb_model=CatBoostRegressor()

In [13]:
catb_cv_model=GridSearchCV(catb_model,params,cv=5,verbose=2).fit(X_train,y_train)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
4:	learn: 323.6911970	total: 10.3ms	remaining: 2.04s
5:	learn: 311.9300928	total: 12.2ms	remaining: 2.01s
6:	learn: 300.2621121	total: 14.1ms	remaining: 2s
7:	learn: 291.2650983	total: 16.3ms	remaining: 2.02s
8:	learn: 281.6923824	total: 18.3ms	remaining: 2.01s
9:	learn: 272.8843676	total: 19.6ms	remaining: 1.94s
10:	learn: 263.7402842	total: 21ms	remaining: 1.89s
11:	learn: 254.3135840	total: 23ms	remaining: 1.9s
12:	learn: 248.0289839	total: 25.2ms	remaining: 1.92s
13:	learn: 238.7489277	total: 27.4ms	remaining: 1.93s
14:	learn: 231.3309018	total: 29.4ms	remaining: 1.93s
15:	learn: 225.3009391	total: 31.4ms	remaining: 1.93s
16:	learn: 218.9395551	total: 33.4ms	remaining: 1.93s
17:	learn: 212.6418512	total: 35.5ms	remaining: 1.94s
18:	learn: 207.0949978	total: 40ms	remaining: 2.06s
19:	learn: 202.3590566	total: 43.9ms	remaining: 2.15s
20:	learn: 196.4389388	total: 48ms	remaining: 2.23s
21:	learn: 190.4074511	total: 51ms	remaining: 2.27s
2

In [15]:
catb_cv_model.best_params_

{'depth': 6, 'iterations': 1000, 'learning_rate': 0.01}

In [16]:
catb_tuned=CatBoostRegressor(depth= 6, iterations=1000, learning_rate=0.01).fit(X_train,y_train)

0:	learn: 363.7199675	total: 1.73ms	remaining: 1.73s
1:	learn: 362.0140103	total: 2.88ms	remaining: 1.44s
2:	learn: 360.5477011	total: 3.96ms	remaining: 1.32s
3:	learn: 358.9212841	total: 5.03ms	remaining: 1.25s
4:	learn: 357.5172937	total: 6.11ms	remaining: 1.22s
5:	learn: 356.1293249	total: 7.16ms	remaining: 1.19s
6:	learn: 354.7101165	total: 8.17ms	remaining: 1.16s
7:	learn: 353.3220938	total: 9.17ms	remaining: 1.14s
8:	learn: 351.8584244	total: 10.2ms	remaining: 1.12s
9:	learn: 350.6144193	total: 11.2ms	remaining: 1.11s
10:	learn: 349.4226939	total: 12.2ms	remaining: 1.1s
11:	learn: 347.9373581	total: 13.3ms	remaining: 1.09s
12:	learn: 346.6082747	total: 14.3ms	remaining: 1.09s
13:	learn: 345.0608062	total: 15.3ms	remaining: 1.08s
14:	learn: 343.7653720	total: 16.4ms	remaining: 1.07s
15:	learn: 342.5314012	total: 17.4ms	remaining: 1.07s
16:	learn: 341.2208170	total: 18.4ms	remaining: 1.06s
17:	learn: 339.9216328	total: 19.5ms	remaining: 1.06s
18:	learn: 338.7681295	total: 20.7ms	re

In [17]:
y_pred=catb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

343.02289410343604